# Code for running Detectron 2 Mask R-CNN on BRACOT Dataset
*Version for Google Colab*

***Install Detectron2 and some other libs***

In [ ]:
# install dependencies: 
!pip install pyyaml==5.1
import torch, torchvision
print(torch.__version__, torch.cuda.is_available())
!gcc --version
# opencv is pre-installed on colab
# install detectron2: (Colab has CUDA 10.1 + torch 1.7)
# See https://detectron2.readthedocs.io/tutorials/install.html for instructions
import torch
assert torch.__version__.startswith("1.7")
!pip install detectron2 -f https://dl.fbaipublicfiles.com/detectron2/wheels/cu101/torch1.7/index.html
# exit(0)  # After installation, you need to "restart runtime" in Colab. This line can also restart runtime

***Connecting to drive (if your data is on GDrive)***

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

***Importing some libs***

In [ ]:
import sys
import os
import matplotlib.pyplot as plt
import numpy as np
import cv2
from PIL import Image
import random
import torchvision

import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog, DatasetCatalog
from detectron2.data.datasets import register_coco_instances
from detectron2.utils.visualizer import ColorMode
from detectron2.engine import DefaultTrainer
from detectron2.config import get_cfg
from detectron2.evaluation import COCOEvaluator, inference_on_dataset
from detectron2.data import build_detection_test_loader
from detectron2 import model_zoo
from google.colab.patches import cv2_imshow


***Defining some important functions***

In [ ]:
# func defining config params
def cfg(train_data, test_data, img_per_batch, base_lr, max_iter):
    cfg = get_cfg()
    cfg.merge_from_file(model_zoo.get_config_file("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml"))
    cfg.DATASETS.TRAIN = (train_data,)
    cfg.DATASETS.TEST = (test_data, )
    cfg.DATALOADER.NUM_WORKERS = 2
    cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml")  
    cfg.SOLVER.IMS_PER_BATCH = img_per_batch
    cfg.SOLVER.BASE_LR = base_lr
    cfg.SOLVER.MAX_ITER = max_iter   
    cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 512   
    cfg.MODEL.ROI_HEADS.NUM_CLASSES = 2 # leaf + background

    return cfg

# func for training
def train(cfg):
    os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)
    trainer = DefaultTrainer(cfg)
    trainer.resume_or_load(resume=False)
    trainer.train()

# return predictor
def def_predictor(cfg, weights_path):
    cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR, weights_path)
    cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.7
    
    return DefaultPredictor(cfg)
    
# func for evaluation on dataset
def eval_coco_dataset(predictor, test_data, cfg):
    evaluator = COCOEvaluator(test_data, cfg, False, output_dir="./output/")
    val_loader = build_detection_test_loader(cfg, test_data)
    inference_on_dataset(predictor.model, val_loader, evaluator)

# func for evaluation on single image
def eval_single_img(cfg, img_path, predictor):
    im = cv2.imread(img_path)
    outputs = predictor(im)
    v = Visualizer(im[:, :, ::-1],
                   metadata=MetadataCatalog.get(cfg.DATASETS.TRAIN[0]), 
                   scale=1, 
                   instance_mode=ColorMode.IMAGE_BW   # remove the colors of unsegmented pixels
    )
    v = v.draw_instance_predictions(outputs["instances"].to("cpu"))
    cv2_imshow(v.get_image()[:, :, ::-1])

# func for returning imgs for further processing in the framework (semantic segmentation + classification)

def create_fig (img, mask):
    temp = img.copy()
    i_, j_, _ = img.shape
    for i in range (0, i_):
        for j in range (0, j_):
            if mask[i][j] < 0.1:
                temp[i][j][0] = 255 
                temp[i][j][1] = 255 
                temp[i][j][2] = 255 
    return temp

def create_outputs(predictor, img_path):
        im = cv2.imread(img_path)
        outputs = predictor(im)
        v = Visualizer(im[:, :, ::-1],
                    scale=1, 
                    instance_mode=ColorMode.IMAGE_BW
        )
        v = v.draw_instance_predictions(outputs["instances"].to("cpu"))
        if not os.path.exists('/content/outputs/'):
            os.makedirs('/content/outputs/instance_seg_result')
        cv2.imwrite(f'/content/outputs/instance_seg_result/instanceseg.png', v.get_image()[:, :, ::-1])

        masks = outputs["instances"].to("cpu").pred_masks.numpy()
        j = 0
        for mask in masks:
            temp = create_fig(im, mask)
            cv2.imwrite(f"/content/outputs/instance_seg_result/{j}.jpg", temp)
            j += 1

### Example

***Registering dataset and defining config***

In [ ]:
# register the datasets giving the json and image root paths
train_json_path = "/content/drive/My Drive/ic2-dataset/train/train_annotation.json"
test_json_path = "/content/drive/My Drive/ic2-dataset/test/test_annotation.json"
train_img_root = "/content/drive/My Drive/ic2-dataset/train/"
test_img_root = "/content/drive/My Drive/ic2-dataset/test/"

train_data = "train_data"
test_data = "test_data"

register_coco_instances(name=train_data, metadata={}, json_file=train_json_path, image_root=train_img_root)
register_coco_instances(name=test_data, metadata={}, json_file=test_json_path, image_root=test_img_root)

# define the config params
cfg_params = cfg(train_data=train_data, test_data=test_data, img_per_batch=1, base_lr=0.005, max_iter=24000)


***Training on dataset***

In [ ]:
# train!
train(cfg=cfg_params)

***COCO evaluation on dataset***

In [ ]:
weights_path="/content/drive/MyDrive/ic2-dataset/model_detectron2_resnet101_100epochs.pth"
predictor = def_predictor(cfg=cfg_params, weights_path=weights_path)
eval_coco_dataset(predictor=predictor, test_data=test_data, cfg=cfg_params)

***Evaluate model on a single image***

In [ ]:
img_path = "/content/drive/MyDrive/ic2-dataset/test/20190831_163222.jpg"
eval_single_img(cfg=cfg_params, img_path=img_path, predictor=predictor)

***Generate outputs for further processing on framework***

In [ ]:
create_outputs(predictor=predictor, img_path=img_path)